In [138]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from datetime import datetime
import pulp

names = ['VIN', 'plant', 'dealer','time']
df = pd.read_csv('Problem_VehicleShipmentRequirement.csv',names=names,header=0,usecols=[0,1,2,3])
names_vdc = ['plant','capacity','rail']
df_vdc = pd.read_csv('existingvdc.csv',names=names_vdc,header=0,usecols=[0,1,2])
names_locations = ['location', 'dealer','lat','long']
df_locations = pd.read_csv('locations.csv',names=names_locations,header=0,usecols=[0,1,2,3])
locations_lp = pd.read_csv('locations.csv', index_col=['Location'])

In [139]:
df_locations['dealer'].value_counts()
dealers = df_locations[df_locations.dealer == 1]
VDCs = df_locations[df_locations.dealer == 0]

In [140]:
dealers_lats = dealers['lat'].tolist()
dealers_longs = dealers['long'].tolist()
dealers_locations = dealers['location'].tolist()

In [141]:
initialROs = df[df.plant == 'RO']
initialFFs = df[df.plant == 'FF']
initial3As = df[df.plant == '3A']
initialSOs = df[df.plant == 'SO']

print(len(initialROs))
print(len(initialFFs))
print(len(initial3As))
print(len(initialSOs))

311842
171269
237191
328273


In [257]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3959 # miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [258]:
#Create list of VDCs with rails

vdcrails = df_vdc['rail'].tolist()
existingvdcs = df_vdc['plant'].tolist()
print(vdcrails)

vdcswithrails = []

for i in range(len(vdcrails)):
    if vdcrails[i]:
        vdcswithrails.append(existingvdcs[i])
        
print(vdcswithrails)
        

[True, True, False, True, True, False, False, True, True, False, False, True, True, True, True, True, True, False, False, False, False, False, False, True, False, False, False, True, False, True, False, True, True, False, False, True, False, False, False, True, True, True, False, True]
['3A', '3F', '7J', '7M', 'BM', 'CE', 'DO', 'DV', 'DW', 'DZ', 'EC', 'FF', 'NZ', 'QT', 'RO', 'RX', 'SO', 'UL', 'VW', 'WL', 'QW', 'WK']


In [259]:
#Instantiate problem class

model = pulp.LpProblem("IEOR 162 final project", pulp.LpMinimize)

In [260]:
#Construct our decision variable lists

VINs = df['VIN'].tolist()
VDCs = df_vdc['plant'].tolist()
dealers_var = dealers['location'].tolist()

capacities = df_vdc['capacity'].tolist()
dealers_3A = initial3As['dealer'].tolist()
dealers_RO = initialROs['dealer'].tolist()
dealers_SO = initialSOs['dealer'].tolist()
dealers_FF = initialFFs['dealer'].tolist()

newdf_3A = initial3As.groupby('dealer').count()
newdf_RO = initialROs.groupby('dealer').count()
newdf_SO = initialSOs.groupby('dealer').count()
newdf_FF = initialFFs.groupby('dealer').count()

vins_3A = newdf_3A['VIN'].tolist()
vins_RO = newdf_RO['VIN'].tolist()
vins_SO = newdf_SO['VIN'].tolist()
vins_FF = newdf_FF['VIN'].tolist()

curr_index_3A = newdf_3A.index.get_values()
curr_index_RO = newdf_RO.index.get_values()
curr_index_SO = newdf_SO.index.get_values()
curr_index_FF = newdf_FF.index.get_values()

new_index_3A = []
new_index_RO = []
new_index_SO = []
new_index_FF = []

for item in curr_index_3A:
    temp = str(item)
    new_index_3A.append(temp)
for item in curr_index_RO:
    temp = str(item)
    new_index_RO.append(temp)
for item in curr_index_SO:
    temp = str(item)
    new_index_SO.append(temp)
for item in curr_index_FF:
    temp = str(item)
    new_index_FF.append(temp)
    
newdf_3A.index = new_index_3A #Setting grouped dataframe's index as dealer
newdf_RO.index = new_index_RO
newdf_SO.index = new_index_SO
newdf_FF.index = new_index_FF
#print(new_index)
#print(newdf)

combined_index = set(new_index_3A + new_index_SO + new_index_RO + new_index_FF)

print(len(dealers_3A))
print(len(dealers_RO))
print(len(dealers_SO))
print(len(dealers_FF))
print(len(dealers_var))

print(len(new_index_3A))
print(len(new_index_SO))
print(len(new_index_RO))
print(len(new_index_FF))

print(len(combined_index))
print(len(set(combined_index)))
#print(dealers_var[:10])

237191
311842
328273
171269
21611
7289
6280
5935
5827
8131
8131


In [261]:
#Instantiate variable classes

truck_vdc = pulp.LpVariable.dicts("# of cars shipped between vdc i and j by truck", ((str(i), str(j)) for i in VDCs for j in VDCs if i != j), lowBound=0, cat='Integer')
truck_dealer = pulp.LpVariable.dicts("# of cars shipped between vdc i and dealer k by truck", ((str(i), str(k)) for i in VDCs for k in dealers_var), lowBound=0, cat='Integer')
arrive_by_truck = pulp.LpVariable.dicts("# of cars arriving at dealer k from vdc i", ((str(i), str(k)) for i in ['3A', 'RO', 'SO', 'FF'] for k in combined_index), lowBound=0, cat='Integer')

rail_vdc = pulp.LpVariable.dicts("# of cars shipped between vdc i and j by rail", ((str(i), str(j)) for i in vdcswithrails for j in vdcswithrails if i != j), lowBound=0, cat='Integer')
rail_dealer = pulp.LpVariable.dicts("# of cars shipped between vdc i and dealer j by rail", ((str(i), str(k)) for i in vdcswithrails for k in dealers_var), lowBound=0, cat='Integer')

In [262]:
#Construct objective function

for i in VDCs:
    model += (pulp.lpSum([(200 + 4*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*(truck_dealer[(str(i), str(j))]) for j in dealers_3A]))
    model += (pulp.lpSum([(200 + 4*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*(truck_vdc[(str(i), str(j))]) for j in VDCs if i != j]))
    
for i in vdcswithrails:
    model += (pulp.lpSum([(100 + (3/20)*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*(rail_vdc[(str(i), str(j))]) for j in vdcswithrails if i != j]))
    model += (pulp.lpSum([(100 + (3/20)*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*(rail_dealer[(str(i), str(j))]) for j in dealers_3A]))
    

/Users/academic/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1349: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [263]:
#Construct constraints

#model += pulp.lpSum([truck_vdc['3A', str(j)] for j in VDCs if str(j) != '3A']) + pulp.lpSum([truck_dealer['3A', str(k)] for k in dealers_3A])  == 237191
#model += pulp.lpSum([truck_dealer['3A', str(k)] + truck_dealer[str(j), str(k)] for k in dealers_3A for j in VDCs if str(j) != '3A']) == 237191

for k in new_index_3A:
    model += (arrive_by_truck[('3A', str(k))] == newdf_3A.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('3A',str(k))])
    model += ((pulp.lpSum([truck_vdc['3A', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != '3A'])) + truck_dealer['3A', str(k)] + (pulp.lpSum([rail_vdc['3A', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != '3A'])) + rail_dealer['3A', str(k)]  == 0)
    


In [264]:
#more constraints

for k in new_index_FF:
    model += (arrive_by_truck[('FF', str(k))] == newdf_FF.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('FF',str(k))])
    model += ((pulp.lpSum([truck_vdc['FF', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'FF'])) + truck_dealer['FF', str(k)] + (pulp.lpSum([rail_vdc['FF', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'FF'])) + rail_dealer['FF', str(k)]  == 0)
    
for k in new_index_RO:
    model += (arrive_by_truck[('RO', str(k))] == newdf_RO.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('RO',str(k))])
    model += ((pulp.lpSum([truck_vdc['RO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'RO'])) + truck_dealer['RO', str(k)] + (pulp.lpSum([rail_vdc['RO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'RO'])) + rail_dealer['RO', str(k)]  == 0)

for k in new_index_SO:
    model += (arrive_by_truck[('SO', str(k))] == newdf_SO.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('SO',str(k))])
    model += ((pulp.lpSum([truck_vdc['SO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'SO'])) + truck_dealer['SO', str(k)] + (pulp.lpSum([rail_vdc['SO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'SO'])) + rail_dealer['SO', str(k)]  == 0)


In [265]:
#stored (inactive constraints)

"""model += ((pulp.lpSum([truck_vdc['RO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'RO'])) + (pulp.lpSum([rail_vdc['RO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'RO']))  == 0)
    model += ((pulp.lpSum([truck_vdc['SO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'SO'])) + (pulp.lpSum([rail_vdc['SO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'SO']))  == 0)
    model += ((pulp.lpSum([truck_vdc['FF', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'FF'])) + (pulp.lpSum([rail_vdc['FF', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'FF']))  == 0)"""
    
#for k in combined_index:
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == (pulp.lpSum([arrive_by_truck[str(i), str(k)] for i in ['3A', 'RO', 'SO', 'FF']])))
    #model += ((pulp.lpSum([truck_vdc[str(i), str(j)] - truck_dealer[str(j), str(k)] for i in ['RO','3A','SO','FF'] for j in VDCs if i != j])) + (pulp.lpSum([rail_vdc[str(i), str(j)] - rail_dealer[str(j), str(k)] for i in ['3A','RO','SO','FF'] for j in vdcswithrails if i != j]))  == 0)
    

"model += ((pulp.lpSum([truck_vdc['RO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'RO'])) + (pulp.lpSum([rail_vdc['RO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'RO']))  == 0)\n    model += ((pulp.lpSum([truck_vdc['SO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'SO'])) + (pulp.lpSum([rail_vdc['SO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'SO']))  == 0)\n    model += ((pulp.lpSum([truck_vdc['FF', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'FF'])) + (pulp.lpSum([rail_vdc['FF', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'FF']))  == 0)"

In [266]:
model.solve()

-1

In [276]:
#pulp.LpStatus[model.status]

In [277]:
#print(pulp.value(model.objective))

In [269]:
output = []
output2 = []
for i, j in truck_vdc:
    var_output = {
        'start': i,
        'mid': j,
        '# of cars': truck_vdc[(i, j)].varValue,
    }
    output.append(var_output)

for i, k in truck_dealer:
    var_output = {
        'start': i,
        'dealer': k,
        '# of cars': truck_dealer[(i, k)].varValue,
    }
    output2.append(var_output)



[{'start': '3A', 'dealer': '2', '# of cars': None}, {'start': '3A', 'dealer': '3', '# of cars': None}, {'start': '3A', 'dealer': '4', '# of cars': None}, {'start': '3A', 'dealer': '5', '# of cars': None}, {'start': '3A', 'dealer': '6', '# of cars': None}, {'start': '3A', 'dealer': '7', '# of cars': None}, {'start': '3A', 'dealer': '8', '# of cars': None}, {'start': '3A', 'dealer': '9', '# of cars': None}, {'start': '3A', 'dealer': '10', '# of cars': None}, {'start': '3A', 'dealer': '11', '# of cars': None}, {'start': '3A', 'dealer': '12', '# of cars': None}, {'start': '3A', 'dealer': '13', '# of cars': None}, {'start': '3A', 'dealer': '14', '# of cars': None}, {'start': '3A', 'dealer': '15', '# of cars': None}, {'start': '3A', 'dealer': '16', '# of cars': None}, {'start': '3A', 'dealer': '17', '# of cars': None}, {'start': '3A', 'dealer': '18', '# of cars': None}, {'start': '3A', 'dealer': '19', '# of cars': None}, {'start': '3A', 'dealer': '20', '# of cars': 0.0}, {'start': '3A', 'dea

In [270]:
start_truck_vdc = []
mid_truck_vdc = []
number_truck_vdc = []
for i, j in truck_vdc:
    start_truck_vdc.append(i)
    mid_truck_vdc.append(j)
    number_truck_vdc.append(truck_vdc[(i, j)].varValue)
    
start_truck_dealer = []
mid_truck_dealer = []
number_truck_dealer = []
for i, j in truck_dealer:
    start_truck_dealer.append(i)
    mid_truck_dealer.append(j)
    number_truck_dealer.append(truck_dealer[(i, j)].varValue)
    
start_rail_vdc = []
mid_rail_vdc = []
number_rail_vdc = []

for i, j in rail_vdc:
    start_rail_vdc.append(i)
    mid_rail_vdc.append(j)
    number_rail_vdc.append(rail_vdc[(i, j)].varValue)
    
start_rail_dealer = []
mid_rail_dealer = []
number_rail_dealer = []
for i, j in rail_dealer:
    start_rail_dealer.append(i)
    mid_rail_dealer.append(j)
    number_rail_dealer.append(rail_dealer[(i, j)].varValue)

In [271]:
df_truck_vdc = pd.DataFrame()
df_truck_vdc['Start of Path'] = start_truck_vdc
df_truck_vdc['End of Path'] = mid_truck_vdc
df_truck_vdc['Number of Cars on Path'] = number_truck_vdc

df_truck_dealer = pd.DataFrame()
df_truck_dealer['Start of Path'] = start_truck_dealer
df_truck_dealer['End of Path'] = mid_truck_dealer
df_truck_dealer['Number of Cars on Path'] = number_truck_dealer

df_rail_vdc = pd.DataFrame()
df_rail_vdc['Start of Path'] = start_rail_vdc
df_rail_vdc['End of Path'] = mid_rail_vdc
df_rail_vdc['Number of Cars on Path'] = number_rail_vdc

df_rail_dealer = pd.DataFrame()
df_rail_dealer['Start of Path'] = start_rail_dealer
df_rail_dealer['End of Path'] = mid_rail_dealer
df_rail_dealer['Number of Cars on Path'] = number_rail_dealer

In [272]:
df_truck_vdc.to_csv('truck_vdc_optimized_routing details2.csv')
df_truck_dealer.to_csv('truck_dealer_optimized_routing details2.csv')
df_rail_vdc.to_csv('rail_vdc_optimized_routing details2.csv')
df_rail_dealer.to_csv('rail_dealer_optimized_routing details2.csv')

In [275]:
#check

for k in new_index_FF:
    print(arrive_by_truck[('FF', str(k))].varValue)

22.0
10.0
57.0
45.0
46.0
53.0
57.0
155.0
6.0
160.0
17.0
51.0
65.0
52.0
16.0
6.0
43.0
10.0
28.0
56.0
23.0
15.0
83.0
3.0
35.0
15.0
108.0
2.0
8.0
5.0
17.0
17.0
24.0
7.0
2.0
42.0
128.0
27.0
9.0
41.0
5.0
3.0
52.0
55.0
121.0
2.0
6.0
16.0
39.0
40.0
36.0
14.0
38.0
38.0
4.0
20.0
30.0
24.0
12.0
8.0
3.0
1.0
1.0
39.0
23.0
10.0
2.0
10.0
3.0
6.0
2.0
43.0
27.0
12.0
4.0
1.0
1.0
26.0
30.0
3.0
6.0
4.0
4.0
6.0
7.0
50.0
25.0
12.0
1.0
73.0
1.0
35.0
68.0
26.0
23.0
38.0
54.0
10.0
13.0
89.0
27.0
73.0
44.0
109.0
70.0
18.0
16.0
44.0
12.0
8.0
30.0
17.0
89.0
22.0
9.0
110.0
31.0
130.0
21.0
155.0
233.0
24.0
19.0
347.0
126.0
76.0
19.0
45.0
107.0
36.0
392.0
133.0
28.0
78.0
15.0
78.0
1.0
58.0
65.0
17.0
40.0
67.0
242.0
49.0
25.0
15.0
26.0
11.0
29.0
16.0
51.0
36.0
10.0
20.0
75.0
120.0
95.0
104.0
85.0
200.0
402.0
184.0
217.0
97.0
232.0
76.0
113.0
78.0
168.0
269.0
192.0
372.0
59.0
165.0
25.0
79.0
25.0
10.0
21.0
88.0
54.0
166.0
188.0
88.0
43.0
13.0
17.0
13.0
29.0
31.0
97.0
1.0
8.0
1.0
162.0
110.0
99.0
204.0
1.0
1.0
1.0
413

20.0
51.0
53.0
161.0
52.0
51.0
32.0
26.0
14.0
9.0
69.0
55.0
105.0
23.0
157.0
131.0
57.0
23.0
33.0
154.0
100.0
88.0
172.0
86.0
48.0
35.0
152.0
41.0
25.0
52.0
209.0
28.0
227.0
45.0
68.0
290.0
419.0
82.0
22.0
101.0
14.0
20.0
119.0
118.0
30.0
294.0
42.0
246.0
52.0
60.0
39.0
68.0
24.0
175.0
42.0
121.0
76.0
101.0
81.0
73.0
79.0
225.0
245.0
238.0
21.0
185.0
61.0
28.0
495.0
241.0
43.0
56.0
355.0
81.0
343.0
90.0
147.0
673.0
80.0
38.0
35.0
63.0
16.0
28.0
178.0
71.0
53.0
260.0
35.0
13.0
81.0
64.0
25.0
38.0
25.0
173.0
54.0
171.0
20.0
91.0
69.0
29.0
14.0
28.0
48.0
74.0
23.0
36.0
53.0
53.0
53.0
19.0
109.0
150.0
38.0
81.0
40.0
22.0
26.0
43.0
41.0
15.0
113.0
121.0
118.0
45.0
11.0
167.0
212.0
52.0
83.0
126.0
23.0
6.0
263.0
82.0
188.0
9.0
10.0
13.0
50.0
21.0
201.0
21.0
44.0
125.0
62.0
143.0
37.0
165.0
39.0
29.0
200.0
64.0
34.0
58.0
22.0
33.0
68.0
133.0
66.0
40.0
124.0
18.0
56.0
11.0
47.0
25.0
35.0
83.0
14.0
227.0
12.0
29.0
34.0
37.0
19.0
67.0
71.0
73.0
97.0
459.0
47.0
34.0
91.0
21.0
68.0
410.0
174.0
98.

147.0
5.0
58.0
37.0
24.0
53.0
13.0
89.0
16.0
87.0
39.0
26.0
266.0
50.0
34.0
64.0
140.0
73.0
43.0
38.0
100.0
41.0
10.0
36.0
28.0
35.0
267.0
24.0
24.0
46.0
42.0
104.0
166.0
43.0
123.0
124.0
130.0
25.0
21.0
33.0
61.0
165.0
49.0
10.0
259.0
11.0
75.0
19.0
18.0
42.0
13.0
45.0
12.0
235.0
35.0
35.0
36.0
22.0
110.0
16.0
93.0
40.0
29.0
24.0
123.0
46.0
117.0
28.0
13.0
17.0
8.0
112.0
63.0
18.0
37.0
42.0
56.0
10.0
154.0
20.0
45.0
129.0
30.0
48.0
57.0
19.0
69.0
12.0
91.0
34.0
16.0
14.0
38.0
6.0
16.0
108.0
53.0
149.0
24.0
87.0
68.0
30.0
172.0
60.0
11.0
54.0
7.0
66.0
100.0
34.0
35.0
39.0
69.0
20.0
21.0
71.0
9.0
34.0
11.0
23.0
46.0
11.0
30.0
36.0
26.0
10.0
68.0
6.0
33.0
32.0
47.0
12.0
74.0
35.0
34.0
17.0
34.0
14.0
68.0
51.0
29.0
72.0
18.0
25.0
63.0
45.0
7.0
17.0
5.0
44.0
44.0
22.0
18.0
194.0
78.0
35.0
61.0
140.0
6.0
15.0
16.0
72.0
22.0
16.0
7.0
21.0
61.0
19.0
22.0
100.0
21.0
26.0
12.0
55.0
127.0
17.0
16.0
41.0
57.0
99.0
11.0
21.0
16.0
40.0
43.0
31.0
31.0
33.0
30.0
67.0
19.0
14.0
17.0
24.0
72.0
73.0
122

In [274]:
for k in

SyntaxError: invalid syntax (<ipython-input-274-59f605870c22>, line 1)